In [8]:
!pip install transformers accelerate sentencepiece datasets tokenizers

In [2]:
!pip install -U transformers tokenizers

In [11]:
!pip install peft bitsandbytes
!pip install -U bitsandbytes

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 모델과 토크나이저 불러오기
model_name = "Toastmachine/LLama3.2_noise_filter"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    trust_remote_code=True,
    device_map="auto"
)

device = next(model.parameters()).device

# 테스트용 노이즈 텍스트
noisy_text = "u노트= 주말 2y만대 개s…|장은 불O 9금 얼룩"  # 예시 텍스트

# 입력 텍스트 준비
prompt = f"### 명령어: 노이즈 텍스트 복원\n\n입력: {noisy_text}\n\n출력:"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# 모델 추론
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=216,
        num_return_sequences=1,
        do_sample=False  # Greedy decoding 사용 (가장 가능성 높은 토큰 선택)
    )

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print("입력 (노이즈 텍스트):", noisy_text)
print("출력 (복원된 텍스트):", generated_text.split("출력:")[-1].strip())

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpect

입력 (노이즈 텍스트): u노트= 주말 2y만대 개s…|장은 불O 9금 얼룩
출력 (복원된 텍스트): 노트북 주말 2만대 개통…이장은 불법 9금 얼룩


In [5]:
# 테스트용 노이즈 텍스트
noisy_text = "프로야구~롯TKIAs광주 경기 y천취소"  # 예시 텍스트

# 입력 텍스트 준비
prompt = f"### 명령어: 노이즈 텍스트 복원\n\n입력: {noisy_text}\n\n출력:"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# 모델 추론
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=216,
        num_return_sequences=1,
        do_sample=False  # Greedy decoding 사용 (가장 가능성 높은 토큰 선택)
    )

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print("입력 (노이즈 텍스트):", noisy_text)
print("출력 (복원된 텍스트):", generated_text.split("출력:")[-1].strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


입력 (노이즈 텍스트): 프로야구~롯TKIAs광주 경기 y천취소
출력 (복원된 텍스트): 프로야구 롯데 KIA광주 경기 3천취소


In [ ]:
import pandas as pd
import torch

# 데이터 불러오기
df = pd.read_csv("/data/ephemeral/data/train.csv")

# text 컬럼의 노이즈 제거
def denoise_text(text):
    # 입력 텍스트 준비
    prompt = f"### 명령어: 노이즈 텍스트 복원\n\n입력: {text}\n\n출력:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # 모델 추론
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=216,
            num_return_sequences=1,
            do_sample=False  # Greedy decoding 사용
        )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text.split("출력:")[-1].strip()

# text 컬럼에 노이즈 제거된 텍스트 적용
df['text'] = df['text'].apply(denoise_text)

# 결과 저장
df.to_csv("/data/ephemeral/data/train_denoised.csv", index=False)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_m

In [5]:
import pandas as pd
import torch

# 데이터 불러오기
df = pd.read_csv("/data/ephemeral/data/train.csv")

# 배치 사이즈 설정
batch_size = 32  # 원하는 배치 사이즈로 조정 가능
total_rows = len(df)
progress_interval = max(1, total_rows // (batch_size * 10))  # 10% 단위로 진행 상태 표시

# 노이즈 제거 함수
def denoise_text(text):
    # 입력 텍스트 준비
    prompt = f"### 명령어: 노이즈 텍스트 복원\n\n입력: {text}\n\n출력:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # 모델 추론
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=216,
            num_return_sequences=1,
            do_sample=False  # Greedy decoding 사용
        )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text.split("출력:")[-1].strip()

# 결과를 저장할 리스트 초기화
cleaned_texts = [None] * total_rows

# 배치 처리를 위한 루프
for batch_num, start_idx in enumerate(range(0, total_rows, batch_size)):
    end_idx = min(start_idx + batch_size, total_rows)
    batch_df = df.iloc[start_idx:end_idx]

    # 각 배치에 대한 노이즈 제거 처리
    for i, text in enumerate(batch_df['text']):
        cleaned_texts[start_idx + i] = denoise_text(text)

    # 진행 상태 표시
    if (batch_num + 1) % progress_interval == 0:
        progress = (end_idx / total_rows) * 100
        print(f"Progress: {progress:.0f}% 완료")

# DataFrame에 결과 적용
df['text'] = cleaned_texts

# 결과 저장
df.to_csv("/data/ephemeral/data/train_denoised_3B.csv", index=False)


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 

# 코드에 따로 추가하였지만 여기서 임시로 None 값 대응 코드 추가

In [5]:
import pandas as pd

# CSV 파일 불러오기
df_denoised = pd.read_csv("/data/ephemeral/data/train_denoised.csv")
df_original = pd.read_csv("/data/ephemeral/data/train.csv")

# None 값 교체
def replace_none_text(row):
    # text가 None이거나 NaN인 경우, 같은 ID의 원본 text로 교체
    if pd.isna(row['text']):
        original_text = df_original.loc[df_original['ID'] == row['ID'], 'text']
        # 원본 텍스트가 존재하면 교체, 그렇지 않으면 그대로 둠
        if not original_text.empty:
            return original_text.values[0]
    return row['text']

# train_denoised의 text 컬럼에서 None을 원본 값으로 교체
df_denoised['text'] = df_denoised.apply(replace_none_text, axis=1)

# 결과 저장
df_denoised.to_csv("/data/ephemeral/data/train_denoised_corrected.csv", index=False)


# 노이즈 복원된 데이터의 ID 만 가지고 이후 리라벨링 할 데이터 목록 추출

## 이경우 LLM 으로 노이즈가 복원된 데이터를 가지고 사용

In [ ]:
import pandas as pd

# 파일 불러오기
df_normal = pd.read_csv("/data/ephemeral/data/noisy_texts_ID.csv")
df_denoised = pd.read_csv("/data/ephemeral/data/train_denoised_test.csv")

# ID 기준으로 매칭 및 필터링
filtered_rows = []

for _, row in df_normal.iterrows():
    text_id = row['ID']
    
    # LLM 으로 노이즈를 어느정도 제거된 텍스트 에서 같은 ID를 가진 행 찾기
    matching_row = df_denoised[df_denoised['ID'] == text_id]
    
    # 일치하는 행이 있고 text 값이 None이 아닌 경우에만 추가
    if not matching_row.empty and pd.notna(matching_row.iloc[0]['text']):
        filtered_rows.append({
            'ID': row['ID'],
            'text': matching_row.iloc[0]['text'],  # train_denoised_none의 text를 사용
            'target': matching_row.iloc[0]['target']
        })

# 결과를 데이터프레임으로 변환
filtered_df = pd.DataFrame(filtered_rows)

# 결과 저장
filtered_df.to_csv("/data/ephemeral/data/filtered_texts_test.csv", index=False)

print(f"총 {len(filtered_df)}개의 텍스트가 필터링되어 저장되었습니다.")